# Pipelines  y   Workflow de un proyecto

# Pipelines y Flujo de Trabajo del Proyecto

## Motivación

La mayoría de los proyectos de ciencia de datos tienen el mismo conjunto de tareas:

1. **ETL**: extraer datos de su fuente, transformarlos y luego cargarlos en una base de datos. Recuerda, **ETL** significa *Extraer*, *Transformar* y *Cargar*.
2. **Enlace**: combinar datos procedentes de diferentes fuentes (también conocido como enlace de registros o coincidencia).
3. **Preprocesar datos**: Esto podría incluir la limpieza y manipulación de datos.
4. **Crear una serie de conjuntos de entrenamiento y validación**.
5. **Crear características**: Recombinar y enriquecer los datos para crear características que ayuden en el trabajo de modelado, así como explorar opciones de imputación.
6. **Entrenar el/los modelo(s)**: Puedes probar diferentes algoritmos, características, y demás.
7. **Selección de modelos**: Evaluar el rendimiento en los conjuntos de validación: Utilizando una métrica adecuada (por ejemplo, $Precisión@k$, $Recall$, $AUC$), examinar el rendimiento de tu modelo "fuera de muestra" basado en la tarea y el contexto de implementación en métricas adecuadas (incluyendo sesgo, precisión, etc.).
8. Pensar en nuevas cosas para probar. Repetir los pasos 1 al 4 según corresponda.

Si la base de código no está estructurada adecuadamente, podrías tener dificultades para recordar los detalles de cada paso una vez que hayas construido algunos modelos. ¿Qué características usaste para cada uno? ¿Qué divisiones de entrenamiento y validación? ¿Qué hiperparámetros?

Tu código también podría estar volviéndose desordenado. ¿Sobrescribiste el código del modelo anterior? Tal vez copiaste, pegaste y editaste código de un modelo anterior. ¿Todavía puedes entender lo que hay ahí? Rápidamente puede convertirse en una mezcolanza que requiere heroísmos para descifrar.

En esta sesión, presentaremos un flujo de trabajo que puede evitar (o al menos reducir) estos problemas.

# Tuberías de Datos

Es útil estructurar los datos en múltiples capas.
En bases de datos, una capa se expresa como esquema.
En la mayoría de los otros formatos, se expresan a través de una estructura de directorio.

##### Crudo
Los datos que recibimos de los socios y fuentes externas son los datos crudos.
Los datos crudos son inmutables. Citando del popular paquete de flujo de trabajo [Data Science Cookiecutter](https://drivendata.github.io/cookiecutter-data-science/#data-is-immutable):

Nunca edites tus datos crudos, especialmente no manualmente y especialmente no en Excel.
No sobrescribas tus datos crudos.
No guardes múltiples versiones de los datos crudos.
Trata los datos (y su formato) como inmutables.
El código que escribas debe mover los datos crudos a través de una tubería hacia tu análisis final.
No deberías tener que ejecutar todos los pasos cada vez que quieras crear una nueva figura (ver Análisis es un DAG), pero cualquiera debería poder reproducir los productos finales con solo el código en src y los datos en data/raw.


##### Intermedio
Si los datos crudos son desordenados, es aconsejable crear una capa intermedia que consista en copias ordenadas de los datos crudos.
Situaciones típicas en las que esto es útil son:
- Se reciben datos en múltiples tipos de archivos diferentes.
- Los campos de datos no están tipados (por ejemplo, archivos csv, Excel) o están mal tipados (fechas como cadenas, formatos de fecha inconsistentes).
- Los nombres de las columnas no son claros, tienen espacios, caracteres especiales o no tienen nombres de columna.

Las transformaciones de crudo a intermedio deben limitarse a corregir los problemas mencionados anteriormente. No debemos combinar diferentes conjuntos de datos ni crear campos calculados. Esto está reservado para la próxima capa.

Los formatos de almacenamiento típicos para la capa intermedia son una base de datos (por ejemplo, `postgres`) o archivos `parquet`.

##### Procesado
Para realizar el trabajo de modelado, los datos de entrada deben combinarse y enriquecerse, por ejemplo, mediante la creación de características.
Los conjuntos de datos creados en este proceso se almacenan en la capa procesada.
A veces puede ser útil dividir esta capa en un modelo de datos de dominio, una capa de características y una capa maestra, pero la estratificación exacta dependerá del contexto del proyecto.

##### Modelos
Los datos procesados se utilizan para entrenar modelos predictivos, modelos explicativos, motores de recomendación y algoritmos de optimización.
Los modelos entrenados se almacenan en la capa de modelos. 
A diferencia de las capas anteriores, los modelos generalmente se almacenan en `pickle` porque no están en formato tabular.

##### Salida del Modelo
Las métricas de rendimiento del modelo, la información de selección de modelos y las predicciones se mantienen en la capa de salida del modelo.

##### Reportes
La generación de informes se puede realizar en todo el conducto.
Por ejemplo, puede haber informes de calidad de datos sobre las entradas, análisis de distribución sobre los datos procesados, predicciones, explicaciones, recomendaciones que se proporcionan al usuario, y evaluación y seguimiento del rendimiento.
Si se construye una interfaz de usuario, accederá a la capa de informes para mostrar información a los usuarios y desarrolladores.
Por ejemplo, un panel de Tableau, Power BI, un cuaderno Jupyter o una salida de Excel leerán desde la capa de informes. En consecuencia, el formato de los datos en la capa de informes se ajustará a la elección de la interfaz.

<img src="images/data_pipeline.png" width="90%">


.png "Data pipeline")

# Configuración del Código
El repositorio de código reflejará la tubería de datos al crear la estructura de carpetas correspondiente para los archivos de Python.

Además, hay varios otros archivos que deben almacenarse y gestionarse.
La comunidad ha llegado a una configuración estándar de los directorios del proyecto que también seguiremos.

Estructura de directorio:

```
├── LICENSE
├── README.md          <- README principal para los desarrolladores que utilizan este proyecto.
├── conf
│   ├── base           <- Espacio para configuraciones compartidas como parámetros.
│   └── local          <- Espacio para configuraciones locales, generalmente credenciales.
│
├── data
│   ├── 01_raw         <- Datos de entrada inmutables
│   ├── 02_intermediate<- Versión limpia de los datos crudos
│   ├── 03_processed   <- Los datos utilizados para el modelado
│   ├── 04_models      <- Modelos entrenados
│   ├── 05_model_output<- Salida del modelo
│   └── 06_reporting   <- Informes y entrada para el frontend
│
├── docs               <- Espacio para la documentación de Sphinx
│
├── notebooks          <- Cuadernos Jupyter. La convención de nombres es la fecha en formato YYYYMMDD (para ordenar), las iniciales del creador y una breve descripción separada por `-`, por ejemplo, `20190601-jqp-exploracion-inicial-de-datos`.
│
├── references         <- Diccionarios de datos, manuales y otros materiales explicativos.
│
├── results            <- Análisis intermedio en formatos como HTML, PDF, LaTeX, etc.
│
├── requirements.txt   <- Archivo de requisitos para reproducir el entorno de análisis, generado con `pip freeze > requirements.txt`.
│
├── .gitignore         <- Evita cargar datos, credenciales, salidas, archivos del sistema, etc.
│
└── src                <- Código fuente para su uso en este proyecto.
    ├── __init__.py    <- Hace que src sea un módulo de Python
    │
    ├── d00_utils      <- Funciones utilizadas en todo el proyecto
    │   └── remove_accents.py
    │
    ├── d01_data       <- Scripts para lectura y escritura de datos, etc.
    │   └── load_data.py
    │
    ├── d02_intermediate<- Scripts para transformar datos de crudos a intermedios
    │   └── create_int_payment_data.py
    │
    ├── d03_processing <- Scripts para convertir datos intermedios en entradas de modelado
    │   └── create_master_table.py
    │
    ├── d04_modelling  <- Scripts para entrenar modelos y luego usar modelos entrenados para hacer predicciones
    │   │
    │   └── train_model.py
    │
    ├── d05_model_evaluation<- Scripts que analizan el rendimiento del modelo y la selección de modelos
    │   └── calculate_performance_metrics.py
    │    
    ├── d06_reporting  <- Scripts para producir tablas de informes
    │   └── create_rpt_payment_summary.py
    │
    └── d06_visualisation<- Scripts para crear gráficos de uso frecuente
        └── visualise_patient_journey.py
```

# Flujo de Trabajo

El flujo de trabajo típico para desarrollar código es el siguiente:

- Prototipo de código en un cuaderno Jupyter.
- Mueva el código a una función que tome datos y parámetros como entradas y devuelva los datos procesados o el modelo entrenado como salida.
- Prueba la función en el cuaderno Jupyter.
- Mueva la función a la carpeta `src`.
- Importa la función en el cuaderno Jupyter.
- Prueba que la función esté funcionando.

Las funciones se pueden importar en un cuaderno de la siguiente manera. Primero, indicamos al cuaderno dónde se encuentran las funciones:

```python
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)
```

Luego, especificamos qué funciones importar:

```python
from d00_utils.my_fun import my_fun
```

¡Pruébalo!

# Tubería de Código

El código que produce las diferentes capas de la tubería de datos debe abstraerse en funciones.

Una *tubería de código* es un conjunto de código que maneja todas las tareas computacionales que necesita tu proyecto desde el principio hasta el final. La tubería más simple es un conjunto de funciones encadenadas.

Por ejemplo:

```python
int_data = create_int_data(raw_data)
pro_drug_features = create_pro_drug_features(int_data)
pro_patient_features = create_pro_patient_features(int_data)
pro_master_table = create_pro_master_table(pro_drug_features, pro_patient_features)
model = train_model(pro_master_table)
rpt_report = produce_report(model)
```

Este es un ejemplo muy esquemático. Normalmente, cada paso se divide en varios subconjuntos creando tuberías para cada capa de la tubería de datos. La tubería de extremo a extremo es la concatenación de las sub-tuberías.

## Ejemplos

#### Ejemplo 1
Aquí tienes un ejemplo simple de una tubería que utiliza el conjunto de datos de Boston de `scikit-learn`:

<img src="images/boston_pipeline.png" width="100%">

Esta tubería tiene dos pasos. El primero, llamado "preprocesamiento", prepara los datos para el modelado creando divisiones de entrenamiento y prueba. El segundo, llamado "modelos, predicciones y métricas", utiliza los datos preprocesados para entrenar modelos, hacer predicciones e imprimir $R^2$ en el conjunto de prueba. La tubería toma entradas (por ejemplo, datos, proporciones de entrenamiento/prueba y tipos de modelo) en un extremo y produce salidas (precisión, por ejemplo) en el otro extremo.

Obviamente, este análisis está incompleto, pero la tubería es un buen comienzo. Dado que usamos el mismo código y datos, podemos ejecutar la tubería de principio a fin y obtener los mismos resultados. Y como dividimos la tubería en funciones, podemos identificar dónde falla la tubería y mejorarla una función a la vez. (Cada función solo necesita usar las mismas entradas y salidas que antes).

También observa la función y los bucles en la segunda parte de la tubería. Somos algo agnósticos sobre los métodos que usamos. Si funciona, ¡

genial! Esta estructura nos permite recorrer muchos tipos de modelos utilizando los mismos datos preprocesados y las mismas predicciones y métricas. Facilita la adición de nuevos métodos y la comparación de resultados, y nos ayuda a concentrarnos en otras partes de la tubería, como la generación de características.

#### Ejemplo 2

La [tubería de la policía](https://github.com/dssg/police-eis), iniciada en [**DSSG 2015**](https://dssg.uchicago.edu/people/2015-fellows-mentors/), es un ejemplo de una tubería relativamente bien desarrollada. Nos permite especificar las opciones de la tubería que deseamos en un archivo yaml, desde el preprocesamiento en adelante. (El código en este repositorio no incluye ETL). Nos proporciona muchas opciones de modelado y facilita las comparaciones.

#### Ejemplo 3

Echa un vistazo a [Triage](http://github.com/dssg/triage), una herramienta que hemos creado para respaldar proyectos similares a DSSG.

## Recursos

* [Nuestra tubería principal](https://github.com/dssg/lead-public), iniciada en DSSG 2014
* [Nuestra tubería de Cincinnati](https://github.com/dssg/cincinnati), iniciada en DSSG 2015
* [Triage](https://github.com/dssg/triage) (una tubería DSSG generalizada)
* [https://scikit-learn.org/stable/modules/compose.html#pipeline]( puedes leer la guia completa de `Pipelines` para `scikit-learnel`)

# Integración de datos

Obtener y conservar datos
Los datos vienen en muchas formas, de muchas fuentes: es posible que obtenga un volcado de base de datos directamente de un socio del proyecto, o puede que necesite extraer datos de la web (consulte Web Scraping básico ). De cualquier manera, una vez que tenga algunos datos en sus manos, deberá incorporarlos a una base de datos y comenzar a formatearlos de tal manera que pueda usarlos para el análisis. Las herramientas de línea de comandos empezarán a resultar útiles aquí. Si sus datos están en un formato similar, CSV estas instrucciones le resultarán útiles . Definitivamente querrá realizar un seguimiento de los pasos que siguió para pasar de datos sin procesar a datos listos para el modelo ( ETL reproducible ).

A menudo, los proyectos de ciencia de datos para el bien social involucran datos confidenciales, por lo que es importante conocer algunos principios básicos de seguridad de datos: Introducción a la seguridad de datos .

## ETL Reproducible

## Motivación

- **Entender lo que hiciste**: Guardar todos los pasos que tomaste para que puedas decir cómo llegaste aquí.
- **Usar lo que hiciste**: Usar y reutilizar el código para este proyecto o para otros. Corregir errores fácilmente. Importar nuevos datos con confianza. ¡HÁZLO IMPLEMENTADO!

Esta sesión se basa en lo que aprendiste la semana pasada en la [sesión CSV a DB](https://github.com/dssg/hitchhikers-guide/tree/master/curriculum/1_getting_and_keeping_data/csv-to-db).

## Posibles Enseñanzas

- Transferencias a bases de datos propietarias (por ejemplo, SQL Server, Oracle).

## Conceptos

Muchas personas asocian la ciencia de datos con algoritmos avanzados de aprendizaje automático, pero la ETL (Extracción, Transformación y Carga) es argumentablemente más importante.

ETL:

1. **Extracción**: Obtener datos de la fuente, por ejemplo, un CSV que te proporcionó un socio.
2. **Transformación**: Llevar los datos al formato que deseas/necesitas, por ejemplo, estandarizar los valores faltantes.
3. **Carga**: Introducir los datos en la base de datos.

Existen dos razones por las que la ETL es tan importante:

1. El resto de tu análisis depende de tu ETL. Por ejemplo, [puedes ignorar algunos de los casos más importantes si eliminas filas con valores faltantes](http://www.stabilityjournal.org/articles/10.5334/sta.cr/).

2. Datos de mejor calidad pueden ayudar más que métodos de mejor calidad.

Por lo tanto, debes hacer la ETL de la siguiente manera:

- Confiabilidad.
- Comprensibilidad.
- Preferiblemente de forma automática.

Herramientas:

- El código es ~~típicamente~~ mejor que las interfaces gráficas de usuario (GUI). --- El código se puede automatizar.
- En igualdad de condiciones, las herramientas de línea de comandos son buenas opciones. Han sido probadas con el tiempo y son eficientes.
- `make` (una herramienta de línea de comandos escrita para compilar software eficientemente).
- Si no puedes guardar el código, guarda las notas, por ejemplo, registra cómo utilizaste Pentaho para transferir una base de datos Oracle a PostgreSQL.


### Pruebas ETL

Corres el riesgo de perder o corromper datos en cada paso. Para asegurarte de que extrajiste, transformaste y cargaste los datos correctamente, puedes realizar comprobaciones simples. Aquí tienes algunas formas:

- Si estás copiando un archivo, verifica el hash antes y después. Deberían coincidir.
- Cuenta las filas. Si deben coincidir antes y después, ¿lo hacen?
- Verifica estadísticas agregadas, como la suma de una columna. Si deben coincidir antes y después, ¿lo hacen?
- Si recibes una copia de base de datos, cuenta el número de esquemas/tablas/secuencias, etc., en las bases de datos de origen y destino. ¿Coinciden? Por ejemplo, solicitamos a los socios que utilizan Oracle que ejecuten el script `count_rows_oracle.sql`, que proporciona el número de filas y columnas en la base de datos de origen. Es una forma (necesaria pero no suficiente) de comprobar que obtuvimos todos los datos.


### ¿Qué pasa si tienes que usar interfaces gráficas de usuario?


# Referencias 
[Esté documento es una adaptación de SSD ](https://github.com/dssg/hitchhikers-guide/tree/master/sources/curriculum/0_before_you_start/pipelines-and-project-workflow)